# YOLO V5

### <u>**Goal**</u>:
Learn to load the yolo v5 model using Dataloop platform
run a train flow for the model

### <u>**Background**</u>:
Yolo V5 model is an extension to the Yolo Family. (V5 was created by 'ultralytics')

the V5 key new added features are part of the train function: using mosaic augmetation and finetuning the anchors
the model is implemented in PyTorch framework

In [ ]:
import datetime
import matplotlib as mpl
# mpl.use('TkAgg')
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import dtlpy as dl
from dtlpy import ml 
from dtlpy.ml import train_utils
# from dtlpy.ml import train_utils
%matplotlib inline 


verbose = True

## Loading

first we will get the dataloop entities

Then we can create our Model Adapter `adapter` using the `model.build()` method.

`adapter` is an instace of the model set to work with our datloop platform.
this contains method like `train()` and `predict_items()` and more usefull methods for building your model using our platform


#### Get Model

In [ ]:
dl.setenv('prod')
project = dl.projects.get('DataloopModels')  
model = dl.models.get('yolo-v5', model_id=None)
model.to_df()
# verbose and model.snapshots.list().to_df()

#### Get the Snapshot

In [ ]:
snapshot = model.snapshots.get(snapshot_name='pretrained-yolo-v5')
verbose and snapshot.to_df()

#### Instanciate the adapter

In [ ]:
# adapter = model.build()
# local develop ==>
adapter = model.build(local_path='/home/shefi/dlp/model_zoo_github/yolov5')

# override cpu
adapter._set_device('cpu')
adapter.logger.handlers[0].setLevel('DEBUG')

adapter.load_from_snapshot(snapshot=snapshot)

## Inference Example

we will use a single item to show how to run inference easily

USER: please choose an item on your own dataset - so you have premission to view and edit it

In [ ]:

item = dl.items.get(item_id='611e174e4c09acc3c5bb81d3')
im_pil = Image.open(item.download())
image = np.asarray(im_pil)

# use the adapter to inference without uploading annotations
all_annotations = adapter.predict_items([item], with_upload=False, min_score=0.1)

item_annotations = all_annotations[0]  # predict items returns a list for each of the input items
print(item_annotations.print(to_return=True))
# print(item.annotations.list().print(to_return=True))
print(item_annotations.to_df())

# add the items to the image
annotated_image = item.annotations.show(image=image, thickness=5)
plt.imshow(annotated_image)

### Prepare Dataset for Training

The "prepare_dataset()" method prepares the raw dataset to be a part of a snapshot and ready for training.  
This will:
1. Clone the dataset to prepare 
2. Create train, validation and test splits
3. Lock the dataset to be read only

We can set the partition split to be random (using float to set the precentage) or we can use DQL filters to set specific items and folder

In [ ]:
fruit_dataset = dl.projects.get('COCO ors').datasets.get('FruitImage')
# # split the dataset to 2 partitions - 80% train 20% validation - randomly
# partitions = {dl.SnapshotPartitionType.TRAIN: 0.8,
#               dl.SnapshotPartitionType.VALIDATION: 0.2}

# use DQL to set the two directories and the train/val split
partitions = {dl.SnapshotPartitionType.TRAIN: dl.Filters(field='dir', values='/train'),
              dl.SnapshotPartitionType.VALIDATION: dl.Filters(field='dir', values='/test')}
    
cloned_dataset = train_utils.prepare_dataset(fruit_dataset,
                                             filters=None,
                                             partitions=partitions)


### Prepare the Snapshot
When running the inferene we used the pretrained `Snapshot`.
    This has the weights of the model saved in a GCS bucket for our global model

Now, when runnig our own specific detector we will create a new `Snapshot`
    * Item bucket - saves the weights to a dedicated item in the dataloop platform
    * Set to specific labels - what are the classes the model will work on
    * Update configuration and other tags etc.

In [ ]:
snapshot_name='fruit-first'
try:
    new_snapshot = model.snapshots.get(snapshot_name=snapshot_name)
except Exception:
    bucket = project.buckets.create(bucket_type=dl.BucketType.ITEM, model_name=model.name, snapshot_name=snapshot_name)
    new_snapshot = snapshot.clone(snapshot_name=snapshot_name,
                                dataset_id=cloned_dataset.id,
                                bucket=bucket,
                                configuration={'batch_size': 2,
                                                'start_epoch': 0,
                                                'max_epoch': 5,
                                                }
                                )

##### Cloned snapshot
Note that the new snapshot was cloned with all it's content.

This means that the Item Bucket was also copied and contains the same stat as the 'pretrained'

This will be true until we will train the model and save out new state.

In [ ]:
new_snapshot.bucket.list_content().to_df()

## Start Training
We are almost ready to train

We will use the adapter methods to run the train:

    1. Load the snapshot to the adapter - contains the dataset to train on, and all the configuraions
    2. Download the dataset locally - saves it to disk
        * use `adapter.convert_from_dtlpy()`  if need to convert the dataloop format
    3. run the train!


In [ ]:
adapter.load_from_snapshot(snapshot=new_snapshot)
root_path, data_path, output_path = adapter.prepare_training()

In [ ]:
print("Training {!r} with snapshot {!r} on data {!r}".format(model.name, new_snapshot.id, data_path))
adapter.train(data_path=data_path, output_path=output_path)

In [ ]:
adapter.save_to_snapshot(local_path=output_path, replace=True)

## Predict New Snapshot on Local Item

We will run a the simple predict.
only this time we will load a different sanpshot to the adapter - the one we just trained and saved

In [ ]:
adapter.load_from_snapshot(snapshot=new_snapshot)

item = dl.items.get(item_id='6110d4a41467ded7a8c2a23d')
image = Image.open(item.download())

all_annotations = adapter.predict_items([item], with_upload=True)  # use with_upload = False
plt.imshow(item.annotations.show(np.asarray(image), thickness=5))